<a href="https://colab.research.google.com/github/pySTEPS/ERAD-nowcasting-course-2022/blob/hands-on-users/hands-on-session-users/notebooks/block_04_deterministic_nowcasts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 3: Nowcasting methods

Now that you know the basics of working with radar data and applying advection schemes, we can continue with the application of different deterministic and probabilistic nowcasting methods. 

In this exercise, we show how to construct, visualize and apply verification metrics to deterministic and probabilistic nowcasts using pysteps.


## Load the data from the previous exercises

First we install pysteps, load and preprocess the example data again by running the helper_nowcasting_methods notebook.

This helper notebook imports the FMI radar data, dBR transforms it and determines the motion field with the Lucas-Kanada optical flow method. The precipitation data is split in a part for forecasting, called `precip_finite`, which is already dBR transformed and not-a-number values have been filled with a minimum value, and a part that will be used as observations (`precip_obs`) for model verification of the nowcasts. 

The metadata corresponding to `precip_finite` is `metadata_dbr` and the metadata of `precip_obs` is `metadata`. 

Finally the motion field variable is called `motion_field`. You can use these variables in these exercises.

In [ ]:
from google.colab import drive
import os
# mount the Google Drive folder
# don't attempt to remount if the drive is already mounted
if not os.path.exists("/content/mnt/MyDrive"):
  drive.mount("mnt")
%cd '/content/mnt/MyDrive/Colab Notebooks'
# run the data notebook to load the input dataset
%run helper_nowcasting_methods.ipynb

# Deterministic nowcasts

In the deterministic nowcasting part, we will use the loaded radar dataset from FMI to create a precipitation nowcast and calculate different verification metrics to assess the skill of the nowcast compared to observations.

The first step is to make a nowcast using the **extrapolation** nowcasting method that simply extrapolates the last observed precipitation field along the motion field. You can follow the example in the [PySTEPS example gallery](https://pysteps.readthedocs.io/en/stable/auto_examples/plot_extrapolation_nowcast.html#sphx-glr-auto-examples-plot-extrapolation-nowcast-py). Calculate the nowcasts for 12 leadtimes, i.e. for 1 hour, and visualize some nowcasts with the observations. The semi-Lagrangian extrapolation method has some keyword arguments that can improve the quality of the nowcast depending on the data. For a full list of the arguments, see the [pySTEPS documentation](https://pysteps.readthedocs.io/en/latest/generated/pysteps.extrapolation.semilagrangian.extrapolate.html).

As rainfall fields can grow and dissipate, a simple extrapolation nowcast probably does not provide sufficient predictability for rainfall forecasts up to a few hours into the future. A first step in this direction was the so-called S-PROG model, a deterministic predecessor of the popular STEPS model (more about that lateR), which makes use of decades of rainfall field research that points out that the lifetime of rainfall cells is related to the spatial extent of these rainfall cells. Hence, larger fields (e.g. stratiform rainfall) have a longer lifetime than smaller rainfall fields (e.g. thunderstorms). S-PROG distinguishes these scales with a Fast Fourier transform and filters out the smallest rainfall fields (with the shortest lifetimes) for increasing lead times. The advantage is that we only put emphasis on the rainfall fields that we can model relatively well, but the disadvantage is that we get a more smoothed rainfall forecast. The probabilistic follow up of this method, STEPS, uses stochastic perturbations to add new small scale rainfall fields to overcome this problem. More about that in the probabilistic nowcasting part. 

Run the code below to construct a simple extrapolation and an S-PROG nowcast and visualize it with the subsequent code block. How good are these rainfall forecasts?


In [ ]:
from matplotlib import pyplot as plt
import numpy as np

from pysteps import nowcasts
from pysteps.visualization import plot_precip_field
%matplotlib inline

# Set nowcast parameters
n_leadtimes = 12

# The extrapolation method has some keyword arguments that can be used to control the nowcasting
extrap_kwargs = {
  "allow_nonfinite_values": False,
  "interp_order": 1,
}

# The extrapolation nowcast
nowcast_method = nowcasts.get_method("extrapolation")
precip_extrap = nowcast_method(
    precip_finite[-1:, :, :].squeeze(),
    motion_field,
    timesteps=n_leadtimes,
    extrap_method="semilagrangian",
    extrap_kwargs=extrap_kwargs,
)

# Back-transform the results from dBR to rain rates
precip_extrap = transformation.dB_transform(
    precip_extrap, 
    threshold=metadata_dbr["threshold"], 
    inverse=True
    )[0]

# Calculate the S-PROG nowcast for comparison
# S-PROG requires as many input fields as 1 + degree of the AR process
# and some other arguments
nowcast_method = nowcasts.get_method("sprog")
precip_sprog = nowcast_method(
    precip_finite[-3:, :, :],
    motion_field,
    timesteps=n_leadtimes,
    n_cascade_levels=6,
    R_thr=metadata_dbr["threshold"],
)

# Back-transform the results from dBR to rain rates
precip_sprog = transformation.dB_transform(
    precip_sprog, 
    threshold=metadata_dbr["threshold"], 
    inverse=True
    )[0]


### Visualize the results

Visualize the observations and the nowcasts for a few lead times. An example on how to do this is provided in [the STEPS nowcast gallery example](https://pysteps.readthedocs.io/en/latest/auto_examples/plot_steps_nowcast.html#stochastic-nowcast-with-steps). You can plot the observations on one row and the corresponding nowcasts below them.

In [ ]:
# Disable warnings
import warnings
warnings.filterwarnings("ignore")

plt.figure(figsize=(16, 12))
# First plot the observations
for i, j in enumerate(range(2, 13, 3)):
    plt.subplot(3, 4, 1 + i)
    plot_precip_field(
        precip_obs[j], 
        geodata=metadata, 
        colorscale="STEPS-NL", 
        colorbar=False
        )
    plt.title(f"Observation at +{(j + 1) * 5} minutes")

# We'll plot the nowcast for four lead times
for i, j in enumerate(range(2, 13, 3)):
    plt.subplot(3, 4, 5 + i)
    plot_precip_field(
        precip_extrap[j], 
        geodata=metadata, 
        colorscale="STEPS-NL", 
        colorbar=False
        )
    plt.title(f"Extrap. nowcast +{(j + 1) * 5} minutes")

for i, j in enumerate(range(2, 13, 3)):
    plt.subplot(3, 4, 9 + i)
    plot_precip_field(
        precip_sprog[j], 
        geodata=metadata, 
        colorscale="STEPS-NL", 
        colorbar=False
        )
    plt.title(f"S-PROG. nowcast +{(j + 1) * 5} minutes")


### Deterministic nowcast verification


After creating the nowcasts, calculate and visualize the probability of detection (POD), false alarm ratio (FAR), critical success index (CSI), and the mean error (ME) for the nowcast as a function of leadtime. This is simple verification that is suitable for deterministic forecasting method. Pysteps actually provides many verification metrics, which can be helpful in your own projects.

Run the code below. How does the S-PROG forecast compare to the simple extrapolation-based forecast?

In [ ]:
from collections import defaultdict
from pysteps import verification


scales = [2, 4, 8, 16, 32, 64, 128, 256, 512]
thr = 1.0
cat_scores = ["POD", "FAR", "CSI"]
cont_scores = ["ME", ]
score = []
score_baseline = []
score_sprog = []
score_steps = []

# Calculate scores for each leadtime
cat_extrap_scores = defaultdict(list)
cat_sprog_scores = defaultdict(list)
cont_extrap_scores = defaultdict(list)
cont_sprog_scores = defaultdict(list)

for i in range(n_leadtimes):
    extrap_score = verification.detcatscores.det_cat_fct(precip_extrap[i], precip_obs[i], thr=thr, scores=cat_scores)
    sprog_score = verification.detcatscores.det_cat_fct(precip_sprog[i], precip_obs[i], thr=thr, scores=cat_scores)

    for name in cat_scores:
        cat_extrap_scores[name].append(extrap_score[name])
        cat_sprog_scores[name].append(sprog_score[name])

    extrap_score = verification.detcontscores.det_cont_fct(precip_extrap[i], precip_obs[i], scores=cont_scores)
    sprog_score = verification.detcontscores.det_cont_fct(precip_sprog[i], precip_obs[i], scores=cont_scores)

    for name in cont_scores:
        cont_extrap_scores[name].append(extrap_score[name])
        cont_sprog_scores[name].append(sprog_score[name])

# Plot scores

plt.figure(figsize=(16, 16))
x = np.arange(1, n_leadtimes + 1) * metadata["accutime"]

plt.subplot(2, 2, 1)
plt.plot(x, cat_extrap_scores["POD"], label="Extrapolation")
plt.plot(x, cat_sprog_scores["POD"], label="S-PROG")
plt.ylim([0, 1])
plt.xlabel("Lead time [min]")
plt.ylabel(f"POD (> {thr} mm/h) ")
plt.title("Probability of detection")
plt.legend()
plt.grid()

plt.subplot(2, 2, 2)
plt.plot(x, cat_extrap_scores["FAR"], label="Extrapolation")
plt.plot(x, cat_sprog_scores["FAR"], label="S-PROG")
plt.ylim([0, 1])
plt.xlabel("Lead time [min]")
plt.ylabel(f"FAR (> {thr} mm/h) ")
plt.title("False Alarm Ratio")
plt.legend()
plt.grid()

plt.subplot(2, 2, 3)
plt.plot(x, cat_extrap_scores["CSI"], label="Extrapolation")
plt.plot(x, cat_sprog_scores["CSI"], label="S-PROG")
plt.ylim([0, 1])
plt.xlabel("Lead time [min]")
plt.ylabel(f"CSI (> {thr} mm/h) ")
plt.title("Critical Success Index")
plt.legend()
plt.grid()

plt.subplot(2, 2, 4)
plt.plot(x, cont_extrap_scores["ME"], label="Extrapolation")
plt.plot(x, cont_sprog_scores["ME"], label="S-PROG")
plt.xlabel("Lead time [min]")
plt.ylabel(f"ME [mm/h] ")
plt.title("Mean Error")
plt.legend()
plt.grid()



### More advanced deterministic nowcasting

As you have noticed by now, S-PROG gives a too smoothed result. It is generally better than simple extrapolation only, especially for longer lead times, but it lacks the possibility of creating new rainfall cells, which makes it not suitable for convective conditions. 

The [LINDA model](https://pysteps.readthedocs.io/en/stable/auto_examples/linda_nowcasts.html#sphx-glr-auto-examples-linda-nowcasts-py) (Lagrangian INtegro-Difference equation model with
Autoregression) in pysteps combines the most state-of-the-art research intothe latest nowcasting methods, such as simple extrapolation, S-PROG, STEPS, ANVIL and cell tracking methods. It combines this with an
integro-difference equation (IDE), which can account for growth and dissipation of rainfall that was already observable in the past radar observations. If growth and dissipation was not observable in the past radar observations, the LINDA model will still not model it, but in other cases this is a great step forward. LINDA is specifically designed for nowcasting intense localized rainfall. For this purpose, it is expected to give better forecast skill than S-PROG or, when we are moving into probabilistic nowcasts, STEPS.

Use the code block below to create a deterministic LINDA nowcast and copy the previous visualization code block to visualize this code block. Note that a forecast with the LINDA method takes longer than using S-PROG. If you have time, you can even use some verification metrics on the forecast. How does this forecast relate to the previous ones?

In [ ]:
# Compute the probabilistic LINDA nowcast
nowcast_linda = nowcasts.linda.forecast(
    precip_fields=precip_finite[-3:, :, :],
    advection_field=motion_field,
    timesteps=n_leadtimes,
    max_num_features=15,
    add_perturbations=False, # This should be set to true for a probabilistic forecast
    num_ens_members=1,
    num_workers=4,
    measure_time=True,
    seed=None,
)[0]

In [12]:
# Plot the forecast here using the visualization code of the previous code blocks.


As you probably have noticed, LINDA has focussed more on the convective cells. For this stratiform example, this may not give the best results. However, this method is worth exploring for more convective rainfall systems and becomes powerful when run in ensemble mode.

## Probabilistic nowcasts

In this part of the exercise, we are basically going to repeat the steps of the deterministic nowcast, but we will construct a probabilistic nowcast with 10 ensemble members and verify this nowcast accordingly. 
If time allows, you can also try to make a LINDA-P (probabilistic) nowcast.

The first step is to make a probablistic nowcast using the STEPS approach that is explained in [the STEPS nowcast gallery example](https://pysteps.readthedocs.io/en/latest/auto_examples/plot_steps_nowcast.html#stochastic-nowcast-with-steps). STEPS is the follow up from the S-PROG model you have used and it extends the scale-dependent nowcasting implementation of S-PROG with a stochastic perturbator that can add and dissipate rainfall per ensemble member. In STEPS, most perturbations will take place on the smallest spatial scales from the fast Fourier transform, as this is the scale where the lifetime of the rainfall fields is lowest and the uncertainty in the forecast is highest. Hence, you can see STEPS as a stochastic way of taking into account the physical processes for rainfall growth and dissipation. 

We are going to make an ensemble nowcast with 10 ensemble members and 12 lead times of 5 min (one hour in total). For a list of all options in the STEPS nowcast, see the [pysteps documentation](https://pysteps.readthedocs.io/en/latest/pysteps_reference/nowcasts.html#pysteps-nowcasts-steps). Run the code blocks below to create and visualize the ensemble nowcasts. Note that this take a little bit longer than the deterministic nowcasts (but note how much faster this is than a typical NWP forecast!). 

What can you say about the quality of these nowcasts compared to just a deterministic nowcasts as you've made earlier?

In [ ]:
# Disable warnings
import warnings
warnings.filterwarnings("ignore")

from matplotlib import pyplot as plt
import numpy as np

from pysteps import nowcasts
from pysteps.postprocessing.ensemblestats import excprob
from pysteps.visualization import plot_precip_field

# Set nowcast parameters
n_ens_members = 10
n_leadtimes = 12
seed = 1234 # None gives a random seed number, but for reproducibility (i.e, 
# every nowcast will give the same perturbations) we set it to a fixed number.

# The STEPS nowcast
nowcast_method = nowcasts.get_method("steps")
precip_forecast = nowcast_method(
    precip_finite[-3:, :, :],
    motion_field,
    timesteps=n_leadtimes,
    n_ens_members=n_ens_members,
    n_cascade_levels=8,
    R_thr=metadata_dbr["threshold"],
    kmperpixel=metadata_dbr["xpixelsize"]/1000.0,
    timestep=metadata_dbr["accutime"],
    noise_method="nonparametric",
    vel_pert_method="bps",
    probmatching_method="cdf",
    mask_method="incremental",
    seed=seed,
    num_workers=4,
)

# Back-transform the results from dBR to rain rates
precip_forecast = transformation.dB_transform(
    precip_forecast, 
    threshold=metadata_dbr["threshold"], 
    inverse=True
    )[0]

### Visualize the result

In [ ]:
plt.figure(figsize=(16, 16))
# First plot the observations
for i, j in enumerate(range(2, 13, 3)):
    plt.subplot(4, 4, 1 + i)
    plot_precip_field(
        precip_obs[j], 
        geodata=metadata, 
        colorscale="STEPS-NL", 
        colorbar=False
        )
    plt.title(f"Observation at +{(j + 1) * 5} minutes")

# We'll plot the ensemble mean for four lead times
precip_forecast_mean = np.mean(precip_forecast[:, :, :, :], axis=0)

plt.figure(figsize=(16, 16))
for i, j in enumerate(range(2, 13, 3)):
    plt.subplot(4, 4, 1 + i)
    plot_precip_field(
        precip_forecast_mean[j], 
        geodata=metadata, 
        colorscale="STEPS-NL", 
        colorbar=False
        )
    plt.title(f"Ensemble mean +{(j + 1) * 5} minutes")

# Then, plot some realizations
plt.figure(figsize=(16, 16))
for i, j in enumerate(range(2, 13, 3)):
    plt.subplot(4, 4, 1 + i)
    plot_precip_field(
        precip_forecast[0,j], 
        geodata=metadata, 
        colorscale="STEPS-NL", 
        colorbar=False
        )
    plt.title(f"Ens. member 1 at +{(j + 1) * 5} minutes")

plt.figure(figsize=(16, 16))
for i, j in enumerate(range(2, 13, 3)):
    plt.subplot(4, 4, 1 + i)
    plot_precip_field(
        precip_forecast[9,j], 
        geodata=metadata, 
        colorscale="STEPS-NL", 
        colorbar=False
        )
    plt.title(f"Ens. member 10 at +{(j + 1) * 5} minutes")

plt.show()



As you can see from the two shown members of the ensemble, the stochastic forecast mantains the same variance as in the observed rainfall field. Hence, it gives a less smoothed outcome than the ensemble mean and also preserves high-intensity rainfall cells.

Another advantage of probabilistic forecasts is that we can visualize the probability of a certain rainfall threshold exceedance. Use the code below to plot such a map and test it for different thresholds.

In [ ]:
# Plot the probability of exceeding 1 mm/h (adjust the code and plot it for other probabilities)

plt.figure(figsize=(16, 10))
for i, j in enumerate(range(2, 13, 3)):
  # Compute exceedence probabilities for a 1.0 mm/h threshold
  P = excprob(precip_forecast[:, j, :, :], 1.0)
  plt.subplot(1, 4, 1 + i)
  plot_precip_field(
      P, 
      geodata=metadata, 
      ptype="prob",
      units="mm/h", 
      probthr=0.5,
      colorbar=False,
      )
  plt.title(f"Exceedance prob. at +{(j + 1) * 5} minutes")

plt.show()

## Ensemble forecast verification
Pysteps includes a number of verification metrics to help users to analyze the general characteristics of the nowcasts in terms of consistency and quality (or goodness). In contrast to the verification of the deterministic nowcast, we have a 10-member ensemble that we want to verify. As every member contains valuable information, it is better not to use the deterministic verification metrics on the ensemble mean, but to use a metric that can take the entire ensemble into account. 

Because of time constraint, we will only plot one metric (the reliability diagram), but if you have time, have a look at the [verification module](https://pysteps.readthedocs.io/en/latest/pysteps_reference/verification.html) of pysteps and implement some metrics yourself.

Run the code block below and have a look at the resulting reliability curves for different thresholds and different lead times (hence, adjust the code yourself).

In [ ]:
from pysteps import verification
from pysteps.postprocessing import ensemblestats

# We start with determining the exceedance probability in the forecast for a
# threshold of 1 mm/h (indicated as 1.0 below) for 1-h lead time (the last lead 
# time in the forecast, indicated as [:, -1, :, :] below).
probability_forecast = ensemblestats.excprob(
    precip_forecast[:, -1, :, :], 
    1.0, 
    ignore_nan=True)

# Reliability diagram
reldiag = verification.reldiag_init(1.0)
verification.reldiag_accum(
    reldiag=reldiag, 
    P_f=probability_forecast, 
    X_o=precip_obs[-1, :, :],
)
fig, ax = plt.subplots()
verification.plot_reldiag(reldiag, ax)
ax.set_title("Reliability diagram (+%i min)" % (n_leadtimes * timestep))
plt.show()

Is this a good result? Ask Jan Verkade for a piece of his knowledge about reliability diagrams. :)

## LINDA-P forecast

If time allows, construct a LINDA-P forecast using the previous code blocks and the starting point below. Also visualize it.

In [ ]:
# Compute the probabilistic LINDA nowcast
nowcast_linda = nowcasts.linda.forecast(
    precip_fields=...,
    advection_field=...,
    timesteps=...,
    max_num_features=15,
    add_perturbations=True,
    num_ens_members=...,
    num_workers=4,
    seed=None,
    measure_time=True,
)[0]

This was a very basic start to nowcasting. We hope that you have a basic understanding of the necessary steps to take, possible pitfalls of the method and the options you have when implementing a nowcasting model for a client. 